In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [72]:
df=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [73]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [75]:
df.shape

(50000, 2)

In [76]:
df['sentiment'].replace({'positive':1,'negative':0},inplace=True)

/tmp/ipykernel_30/4240984564.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['sentiment'].replace({'positive':1,'negative':0},inplace=True)
/tmp/ipykernel_30/4240984564.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'].replace({'positive':1,'negative':0},inplace=True)


In [77]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [78]:
#removing punctution
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [79]:
def remove_punctuation(text):
    punctuationfree=''.join([i for i in text if i not in string.punctuation])
    return punctuationfree
df['clean_msg']=df['review'].apply(lambda x: remove_punctuation(x))

In [80]:
# lower case
df['msg_lower']=df['clean_msg'].apply(lambda x: x.lower())

In [81]:
# tokenization
import re
def tokenization(text):
    token=re.split('W+',text)
    return token

df['tokenized_msg']=df['msg_lower'].apply(lambda x: tokenization(x))

In [82]:
# removal of stopword
import nltk
stopword=nltk.corpus.stopwords.words('english')


def remove_stopword(text):
    output=[i for i in text if i not in stopword]
    return output


df['without_stopword']=df['tokenized_msg'].apply(lambda x : remove_stopword(x))

In [83]:
nltk.download('wordnet')
nltk.download("stopwords")  

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [84]:
#lemmitization
from nltk.stem import WordNetLemmatizer
word_lemm =WordNetLemmatizer()


def lemmatization(text):
    text_lemm=[word_lemm.lemmatize(i) for i in text]
    return text_lemm



In [86]:
nltk.download('wordnet')
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
replace /usr/share/nltk_data/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [87]:
df['lemm_msg']=df['without_stopword'].apply(lambda x: lemmatization(x))

In [88]:
x=df['sentiment']
df['lemm_msg'] = df['lemm_msg'].apply(lambda x: ''.join(str(x)) if isinstance(x, (list, str)) else str(x))


In [89]:
x = df['lemm_msg'].astype(str)


In [90]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
vect=CountVectorizer(ngram_range=(1,1),min_df=5,max_features=8000)
x_vect=vect.fit_transform(x).toarray()
y=df['sentiment']

In [91]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_vect,y,test_size=0.2)

In [92]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
clf1=GaussianNB()
clf2=MultinomialNB()
clf3=BernoulliNB()

In [93]:
clf1.fit(x_train,y_train)
clf2.fit(x_train,y_train)
clf3.fit(x_train,y_train)

BernoulliNB()

In [94]:
y_pred1=clf1.predict(x_test)
y_pred2=clf2.predict(x_test)
y_pred3=clf3.predict(x_test)

In [95]:
from sklearn.metrics import accuracy_score
print('Gaussian',accuracy_score(y_test,y_pred1))
print('Multinomial',accuracy_score(y_test,y_pred2))
print('Bernoulli',accuracy_score(y_test,y_pred3))


Gaussian 0.7475
Multinomial 0.8342
Bernoulli 0.8474


**Bernoulli Naive Bayes achieved the highest accuracy (0.8474), followed by Multinomial (0.8342) and Gaussian (0.7475).**
**

**Trying SVM and KNN algorithms to compare their accuracy**

In [58]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
knn.fit(x_train,y_train)

KNeighborsClassifier()

In [97]:
y_pred=knn.predict(x_test)
accuracy_score(y_test,y_pred)

0.513

In [61]:
from sklearn.svm import SVC
svm_model=SVC()
svm_model.fit(x_train,y_train)

SVC()

In [98]:
y_pred=svm_model.predict(x_test)
accuracy_score(y_test,y_pred)

0.5182

In [ ]:
Both SVM and KNN performed poorly compared to other models, showing lower accuracy scores.


**Both SVM and KNN performed poorly compared to other models, showing lower accuracy scores****